数据预处理部分，直接运行就可以，只需要修改路径即可

In [1]:
import numpy as np
from dataload import preprocess
pre=preprocess()
dataset,testset=pre.preprocessingdata_news()

C:\ProgramData\Anaconda3\envs\py38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\envs\py38\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\envs\py38\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


正在数据集删除空数据并分词....


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\14433\AppData\Local\Temp\jieba.cache
Loading model cost 0.474 seconds.
Prefix dict has been built successfully.


正在删除测试集空数据并分词....


In [2]:
dataset_list,labelset,testset_list=pre.processingbert(dataset,testset)

In [11]:
from fastNLP import DataSet
data_init=DataSet({'raw_chars':dataset_list,'target':labelset})
data_train=data_init[:30000]
data_validation=data_init[30001:35000]
data_test=DataSet({'raw_chars':testset_list})
from fastNLP import Vocabulary
vocab=Vocabulary()
vocab.from_dataset(data_train,field_name='raw_chars',no_create_entry_dataset=[data_validation])
vocab.index_dataset(data_train,field_name='raw_chars')
vocab.index_dataset(data_validation,field_name='raw_chars')
vocab.index_dataset(data_test,field_name='raw_chars')
target_vocab=Vocabulary(padding=None,unknown=None)
target_vocab.from_dataset(data_train,field_name='target')
target_vocab.index_dataset(data_train,field_name='target')
target_vocab.index_dataset(data_validation,field_name='target')

Vocabulary([0, -1, 1]...)

In [13]:
data_train.rename_field('raw_chars','words')
data_validation.rename_field('raw_chars','words')
data_train.set_input('words')
data_train.set_target('target')
data_validation.set_input('words')
data_validation.set_target('target')

KeyError: 'DataSet has no field named raw_chars.'

In [10]:
import torch
from fastNLP.embeddings import BertEmbedding
from fastNLP import Vocabulary
embed=BertEmbedding(vocab,model_dir_or_name='cn-wwm', include_cls_sep=True,auto_truncate=True)
from fastNLP.models import BertForSequenceClassification
model = BertForSequenceClassification(embed,num_labels=3)

ImportError: cannot import name 'BertEmbedding' from 'fastNLP.embeddings' (C:\ProgramData\Anaconda3\envs\py38\lib\site-packages\fastNLP\embeddings\__init__.py)

In [19]:
from fastNLP import Trainer, CrossEntropyLoss, AccuracyMetric, Adam
device = 0 if torch.cuda.is_available() else 'cpu'
print(device)
trainer = Trainer(data_train, model,
                  optimizer=Adam(model_params=model.parameters(), lr=2e-5),
                  loss=CrossEntropyLoss(), device=device,
                  batch_size=8,dev_data=data_validation,
                  metrics=AccuracyMetric(), n_epochs=5, print_every=1)
trainer.train()

In [21]:
def batch_predict(model,data,device):
    submission=pd.DataFrame(columns=['Prediction'])
#    submission = pd.DataFrame(columns=['ID','Prediction'])
    submission=[]
    for i in range(len(data)):
    #for i in range(5):
#         print(data.words[i])
        if (i%100==0):print(f"第{i}次预测完成。")
        model=model.to(device)
        tensor = torch.tensor(data['raw_chars'][i]).to(device)
        pred = model.predict(tensor.view(1,-1))
        prob = pred['pred'].cpu().numpy()[0]
        submission.append(prob)
    return submission
submission=batch_predict(model,data_test,device)


In [23]:
submission2=[]
for i in range(len(submission)):
    if(submission[i]==2):submission2.append(0)
    elif(submission[i]==0):submission2.append(1)
    elif(submission[i]==1):submission2.append(-1)
    else:submission2.append(submission[i])

np.savetxt('.//result//bert.txt',submission2,fmt='%d',delimiter='\n')
print("OK")